In [375]:
import numpy as np
import cv2 as cv
import polars as pl
import os

In [376]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [377]:
img = cv.imread('/content/drive/My Drive/IMG_0536.JPG')
assert img is not None, "file could not be read, check with os.path.exists()"

In [378]:
#img
kernal = np.ones((3,3),np.uint8)
kernal = kernal / 3
kernalWidth, kernalHeight = kernal.shape

In [379]:
height,width,channels = img.shape
print(height,width,channels)

2048 1925 3


In [380]:
sizeX = round(height/kernalWidth)
sizeY = round(width/kernalHeight)
print(sizeX,sizeY)
print(f"Times kernal should be slid on the X axis {sizeX}")
print(f"Times kernal should be slid on the Y axis {sizeY}")

683 642
Times kernal should be slid on the X axis 683
Times kernal should be slid on the Y axis 642


In [381]:
poolMatrix = np.zeros((sizeX,sizeY))

In [382]:
def blur(img,kernal):
  height,width,channels = img.shape
  X_size, Y_size = kernal.shape
  for i in range(0, width, X_size):
    for j in range(0, height, Y_size):

      x_end = min(i + X_size, width) #ensures the slices isn't wider than the image itselft
      y_end = min(j + Y_size, height) #ensures the slices isn't taller than the image itselft
      segment = img[j:y_end, i:x_end]
      outline = segment @ kernal
      max_value = np.max(outline)
      outline = np.where(outline == max_value, max_value, 0)
      img[j:y_end, i:x_end] = outline

  return img


In [383]:
def maxPool(img,kernal,pool):
  height,width,channels = img.shape
  X_size, Y_size = kernal.shape
  for i in range(height // X_size):
    for j in range(width // Y_size):
      segment = img[i*X_size:(i+1)*X_size,j*Y_size:(j+1)*Y_size]
      max_value = np.max(segment)
      pool[i,j] = max_value
  return pool

In [384]:
def minPool(img,kernal,pool):
  height,width,channels = img.shape
  X_size, Y_size = kernal.shape
  for i in range(height // X_size):
    for j in range(width // Y_size):
      segment = img[i*X_size:(i+1)*X_size,j*Y_size:(j+1)*Y_size]
      min_value = np.min(segment)
      pool[i,j] = min_value
  return pool

In [385]:
cv.imwrite("original.jpg", img)

True

In [386]:
pool = maxPool(img,kernal,poolMatrix)
cv.imwrite("maxpooled.jpg", pool)

True

In [389]:
pool = minPool(img,kernal,poolMatrix)
cv.imwrite("minpooled.jpg", pool)

True

In [388]:
outputBlur = blur(img,kernal)
cv.imwrite("blurredImage.jpg", outputBlur)

True